In [1]:
from urllib.request import urlopen, urlretrieve
from os.path import exists, join, isdir 
from os import makedirs
import sys
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
from tqdm import tqdm
sys.path.append('../archive_digger/')
import archive_digger as ad

In [2]:
df = ad.get_harps_database(clobber=False)

Loaded: ../data/harps_db.csv


In [3]:
tois = ad.get_tois(clobber=False)
tois.shape

Loaded: ../data/TOIs.csv


(6160, 62)

In [ ]:
outdir = '../all_tois/v02/'
tois_unique = tois.drop_duplicates(subset='TIC ID')
for i,row in tqdm(tois_unique[::-1].iterrows()):
    # print(row)
    ra, dec = row['RA'], row['Dec']
    p1,p2 = str(row.TOI).split('.')
    toiid = p1.zfill(4)
    name = 'TOI'+toiid
    target_coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
    res = ad.query_target(target_coord, df, dist=10*u.arcsec, verbose=False)
    if isinstance(res, pd.DataFrame):
        print(res['Target'].values[0])
        if not isdir(outdir):
            makedirs(outdir)

        ad.plot_fov(target_coord,res,name=name,fov_rad=60*u.arcsec,ang_dist=15*u.arcsec,
                 survey='DSS',verbose=True,outdir=outdir,savefig=True)

## results

In [36]:
from glob import glob

fl = glob('../all_tois/v02/*')

toiids, names, exofops, harps = [],[],[],[]
for f in sorted(fl):
    toi,name = f.split('/')[-1].split('_')
    base = 'https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url='
    harps_url = base+f'../Ver_02/{name}_RVs/{name}_HARPS_RVBank_ver02_rvsonly.dat&run'
    exofop_url = f'https://exofop.ipac.caltech.edu/tess/target.php?id=TOI-{toi[3:]}'
    toiids.append(toi)
    names.append(name)
    exofops.append(exofop_url)
    harps.append(harps_url)
    print(toi,exofop_url,harps_url)

TOI0101 https://exofop.ipac.caltech.edu/tess/target.php?id=TOI-0101 https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/WASP-46_RVs/WASP-46_HARPS_RVBank_ver02_rvsonly.dat&run
TOI0102 https://exofop.ipac.caltech.edu/tess/target.php?id=TOI-0102 https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/WASP-62_RVs/WASP-62_HARPS_RVBank_ver02_rvsonly.dat&run
TOI0116 https://exofop.ipac.caltech.edu/tess/target.php?id=TOI-0116 https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/WASP-91_RVs/WASP-91_HARPS_RVBank_ver02_rvsonly.dat&run
TOI0118 https://exofop.ipac.caltech.edu/tess/target.php?id=TOI-0118 https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/HD219666_RVs/HD219666_HARPS_RVBank_ver02_rvsonly.dat&run
TOI0119 https://exofop.ipac.caltech.edu/tess/target.php?id=TOI-0119 https://www2.mpia-hd.mpg.de/homes/trifonov/GLS/gls.html?fend=1&url=../Ver_02/HIP31609_RVs/HIP31609_HARPS_RVBank_ver02_rvsonly.dat&

In [37]:
df_url = pd.DataFrame(np.c_[toiids,names,exofops,harps],columns=['toi','name','exofop_url','harps_url'])
df_url

,toi,name,exofop_url,harps_url
0,TOI0101,WASP-46,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
1,TOI0102,WASP-62,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
2,TOI0116,WASP-91,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
3,TOI0118,HD219666,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
4,TOI0119,HIP31609,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
...,...,...,...,...
271,TOI6806,TYC8459-487-1,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
272,TOI6833,K2-138,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
273,TOI6835,K2-141,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...
274,TOI6965,HD50554,https://exofop.ipac.caltech.edu/tess/target.ph...,https://www2.mpia-hd.mpg.de/homes/trifonov/GLS...


In [39]:
df_url.to_csv('../data/toi_with_harps_v2.csv',index=False)